In [ ]:
# check versions and devices
print("python version: ", sys.version)
# print("tf version: ", tf.__version__)
# print("devices: ", tf.config.list_physical_devices())

In [ ]:
# load dataset
import tensorflow_datasets as tfds
import tensorflow as tf
(train_ds, test_ds), ds_info = tfds.load(
    'cars196',
    split=['train[:80%]+test[:80%]', 'train[80%:]+test[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)
print("Num train samples: ", tf.data.experimental.cardinality(train_ds).numpy())
print("Num test samples: ", tf.data.experimental.cardinality(test_ds).numpy())

In [ ]:
# show some samples
tfds.visualization.show_examples(train_ds, ds_info, rows=1, cols=4)

In [ ]:
# normalize
height = width = 224
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x,(height, width)), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x,(height, width)), y))

In [ ]:
# pipeline
autotune = tf.data.AUTOTUNE
batch_size = 128

def augment(image, label):
    image = tf.image.resize_with_crop_or_pad(image, height + 6, width + 6)
    image = tf.image.random_crop(image, size=[height, width, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, 0.2)
    image = tf.image.random_contrast(image, 0.5, 2)
    image = tf.image.random_saturation(image, 0, 2)
    return image, label

train_ds = train_ds.cache().map(augment).shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=autotune)
test_ds = test_ds.cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=autotune)

In [ ]:
# model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(196)
])

model.summary()

In [ ]:
# compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# train
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=3
)

In [ ]:
# save and load model
from tensorflow.keras.models import load_model
import pandas as pd

hist_df = pd.DataFrame(history.history)
# save model and history
model.save('model.h5')
with open("hist.json", "w") as f:
    hist_df.to_json(f)

# load model and history
model = load_model('model.h5')
model_hist = pd.read_json('hist.json')

In [ ]:
print("Average training accuracy: ", model_hist['accuracy'].mean()*100, "%")
print("Average validation accuracy: ", model_hist['val_accuracy'].mean()*100, "%")
print("Average training loss: ", model_hist['loss'].mean())
print("Average validation loss: ", model_hist['val_loss'].mean())

In [ ]:
# plot accuracy and val_accuracy
import matplotlib.pyplot as plt
plt.plot(model_hist['accuracy'], label='train')
plt.plot(model_hist['val_accuracy'], label = 'test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc ='upper left')
plt.ylim([0.0, 1.0])
plt.show()